In [38]:
# Import dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from selenium import webdriver
from sklearn.model_selection import train_test_split

In [2]:
# Set up Splinter
browser = Browser('chrome')

# Solution for multiple pages

###Scraping

In [3]:
# Create a list to hold data from all pages
all_page_data = []

for i in range(1, 32):
    url = f'https://www.canadiantire.ca/en/promotions/hot-sale/home-pet.html?page={i}'
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    # Extract the data for this page
    nameofthegoods = soup.find_all('div', class_='nl-product-card__title nl-product--trim')
    new_price = soup.find_all('div', class_='nl-price--charge')
    old_price = soup.find_all('div', class_='nl-price--was')
    stars = soup.find_all(class_='bv_averageRating_component_container')
    observations = soup.find_all(class_='bv_numReviews_component_container')

    # Ensure all lists have the same length
    min_length = min(len(nameofthegoods), len(new_price))
    
    # Iterate over the items and add them to the page_data list
    page_data = []
    for j in range(min_length):
        item_data = {
            'name': nameofthegoods[j].text if j < len(nameofthegoods) else None,
            'new_price': new_price[j].text if j < len(new_price) else None,
            'old_price': old_price[j].text if j < len(old_price) else None,
            'stars': stars[j].text if j < len(stars) else None,
            'observations': observations[j].text if j < len(observations) else None
        }
        page_data.append(item_data)

    # Append the page data to the overall data list
    all_page_data.extend(page_data)


In [68]:

cantire_df_RAW = pd.DataFrame(all_page_data)

# Clean and Display the DataFrame
cantire_df_RAW['observations'] = cantire_df_RAW['observations'].apply(
    lambda x: x.strip().replace('(', '').replace(')', '') if x is not None else None
)
cantire_df_RAW['old_price'] = cantire_df_RAW['old_price'].str.extract(r'(\$\d{1,3}(?:,\d{3})*(?:\.\d{2})?)')[0]
cantire_df_RAW['new_price'] = cantire_df_RAW['new_price'].str.replace('From ', '')
cantire_df_RAW.shape

(744, 5)

In [18]:
cantire_df_RAW.to_csv('cantire_df_RAW.csv', index=False)
cantire_df_RAW

,name,new_price,old_price,stars,observations
0,"For Living 30 Pint 2-Speed Dehumidifier, Bucke...",$219.99,$279.99,4.4,182.0
1,For Living 6-ft Portable Plastic & Metal Foldi...,$59.99,$79.99,4.3,260.0
2,Tineco iCarpet Portable Lightweight Upright Ca...,$149.99,$499.99,4.3,61.0
3,Sauder 3-Door Wardrobe/Armoire Clothes Storage...,$279.99,$359.99,4.0,32.0
4,iRobot® Roomba® 691 Robot Vacuum – Self Chargi...,$229.99,$499.99,4.2,222.0
...,...,...,...,...,...
739,NOMA Indoor Single Tube LED Light Bar with Mot...,$82.49,$164.99,3.7,9.0
740,PADERNO Richmond Satin Stainless Steel Flatwar...,$49.99,$129.99,4.4,39.0
741,OttLite Task Flashlight & Clip Lamp,$29.99,$39.99,3.7,3.0
742,Smart Tiles Peel & Stick Self-Adhesive Kitchen...,$24.49,$34.99,4.9,637.0


In [49]:
# Shortcut without scraping 31 pages
# cantire_df_RAW=pd.read_csv('cantire_df_RAW.csv')

In [50]:
# Remove commas from 'old_price' and convert to float
# Remove unwanted characters and convert to float
cantire_df_RAW['old_price'] = cantire_df_RAW['old_price'].str.replace('[\$,]', '', regex=True).astype(float)
cantire_df_RAW['new_price'] = cantire_df_RAW['new_price'].str.replace('[\$,]', '', regex=True).astype(float)

# Calculate savings and store in a new column
cantire_df_RAW['Savings'] = cantire_df_RAW['old_price'] - cantire_df_RAW['new_price']
cantire_df_RAW['discount']=(1-(cantire_df_RAW['new_price']/cantire_df_RAW['old_price'])).round(2)
cantire_df_RAW

,name,new_price,old_price,stars,observations,Savings,discount
0,"For Living 30 Pint 2-Speed Dehumidifier, Bucke...",219.99,279.99,4.4,182.0,60.0,0.21
1,For Living 6-ft Portable Plastic & Metal Foldi...,59.99,79.99,4.3,260.0,20.0,0.25
2,Tineco iCarpet Portable Lightweight Upright Ca...,149.99,499.99,4.3,61.0,350.0,0.70
3,Sauder 3-Door Wardrobe/Armoire Clothes Storage...,279.99,359.99,4.0,32.0,80.0,0.22
4,iRobot® Roomba® 691 Robot Vacuum – Self Chargi...,229.99,499.99,4.2,222.0,270.0,0.54
...,...,...,...,...,...,...,...
739,NOMA Indoor Single Tube LED Light Bar with Mot...,82.49,164.99,3.7,9.0,82.5,0.50
740,PADERNO Richmond Satin Stainless Steel Flatwar...,49.99,129.99,4.4,39.0,80.0,0.62
741,OttLite Task Flashlight & Clip Lamp,29.99,39.99,3.7,3.0,10.0,0.25
742,Smart Tiles Peel & Stick Self-Adhesive Kitchen...,24.49,34.99,4.9,637.0,10.5,0.30


DROP NA AND NoN

In [47]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split


In [51]:
# Create index based on 'name' column
reduced_cantire = cantire_df_RAW.set_index('name')
# Drop rows with NaN or None values
reduced_cantire.dropna(inplace=True)
reduced_cantire

,new_price,old_price,stars,observations,Savings,discount
name,,,,,,
"For Living 30 Pint 2-Speed Dehumidifier, Bucket/Continuous Drain, ENERGY STAR® Certified, White",219.99,279.99,4.4,182.0,60.0,0.21
"For Living 6-ft Portable Plastic & Metal Folding Table with Handle, Grey",59.99,79.99,4.3,260.0,20.0,0.25
Tineco iCarpet Portable Lightweight Upright Carpet Vacuum Cleaner,149.99,499.99,4.3,61.0,350.0,0.70
"Sauder 3-Door Wardrobe/Armoire Clothes Storage Cabinet With Hanger Rod & Shelves, Cinnamon",279.99,359.99,4.0,32.0,80.0,0.22
"iRobot® Roomba® 691 Robot Vacuum – Self Charging, Wi-Fi Connected, Good for Pet Hair",229.99,499.99,4.2,222.0,270.0,0.54
...,...,...,...,...,...,...
"NOMA Indoor Single Tube LED Light Bar with Motion Sensor, 5000 Lumens, 4-ft, Daylight, 50W",82.49,164.99,3.7,9.0,82.5,0.50
"PADERNO Richmond Satin Stainless Steel Flatware Set, Serves 4, 20-pc",49.99,129.99,4.4,39.0,80.0,0.62
OttLite Task Flashlight & Clip Lamp,29.99,39.99,3.7,3.0,10.0,0.25


In [52]:
def better_dataframe(reduced_cantire):
    # Drop the 'new_price' column correctly
    X = reduced_cantire.drop('stars', axis=1)
    y = reduced_cantire['stars'].values.reshape(-1, 1)

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=0)

    # Train the model
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = model.predict(X_test)

    # Calculate Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_test, y_pred)
    print("Mean Absolute Error:", mae)




In [53]:
# Call the function with the DataFrame
better_dataframe(reduced_cantire)

Mean Absolute Error: 0.39843545688545695


/Users/zhibekabdyramanova/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Imputation

In [56]:
from sklearn.impute import SimpleImputer

# Fill in the lines below: imputation
my_imputer = SimpleImputer()
imputation = cantire_df_RAW.copy()  # Create a copy to avoid modifying the original DataFrame
imputation.set_index('name', inplace=True)  # Set the index to 'name'

# Impute missing values
imputed_cantire = pd.DataFrame(my_imputer.fit_transform(imputation), columns=imputation.columns, index=imputation.index)
imputed_cantire.shape                              

(744, 6)

In [29]:
better_dataframe(imputed_cantire)

/Users/zhibekabdyramanova/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Mean Absolute Error: 0.3384829138867086


In [58]:
from sklearn.impute import SimpleImputer

# Fill in the lines below: imputation
my_imputer = SimpleImputer(strategy='median')
alternative_imputation = cantire_df_RAW.copy()  # Create a copy to avoid modifying the original DataFrame
alternative_imputation.set_index('name', inplace=True)  # Set the index to 'name'

# Impute missing values
alternative_imputed_cantire = pd.DataFrame(my_imputer.fit_transform(imputation), columns=imputation.columns, index=imputation.index)
alternative_imputed_cantire

,new_price,old_price,stars,observations,Savings,discount
name,,,,,,
"For Living 30 Pint 2-Speed Dehumidifier, Bucket/Continuous Drain, ENERGY STAR® Certified, White",219.99,279.99,4.4,182.0,60.0,0.21
"For Living 6-ft Portable Plastic & Metal Folding Table with Handle, Grey",59.99,79.99,4.3,260.0,20.0,0.25
Tineco iCarpet Portable Lightweight Upright Carpet Vacuum Cleaner,149.99,499.99,4.3,61.0,350.0,0.70
"Sauder 3-Door Wardrobe/Armoire Clothes Storage Cabinet With Hanger Rod & Shelves, Cinnamon",279.99,359.99,4.0,32.0,80.0,0.22
"iRobot® Roomba® 691 Robot Vacuum – Self Charging, Wi-Fi Connected, Good for Pet Hair",229.99,499.99,4.2,222.0,270.0,0.54
...,...,...,...,...,...,...
"NOMA Indoor Single Tube LED Light Bar with Motion Sensor, 5000 Lumens, 4-ft, Daylight, 50W",82.49,164.99,3.7,9.0,82.5,0.50
"PADERNO Richmond Satin Stainless Steel Flatware Set, Serves 4, 20-pc",49.99,129.99,4.4,39.0,80.0,0.62
OttLite Task Flashlight & Clip Lamp,29.99,39.99,3.7,3.0,10.0,0.25


In [59]:
better_dataframe(alternative_imputed_cantire)

Mean Absolute Error: 0.3442819409013601


/Users/zhibekabdyramanova/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [61]:
reduced_cantire

,new_price,old_price,stars,observations,Savings,discount
name,,,,,,
"For Living 30 Pint 2-Speed Dehumidifier, Bucket/Continuous Drain, ENERGY STAR® Certified, White",219.99,279.99,4.4,182.0,60.0,0.21
"For Living 6-ft Portable Plastic & Metal Folding Table with Handle, Grey",59.99,79.99,4.3,260.0,20.0,0.25
Tineco iCarpet Portable Lightweight Upright Carpet Vacuum Cleaner,149.99,499.99,4.3,61.0,350.0,0.70
"Sauder 3-Door Wardrobe/Armoire Clothes Storage Cabinet With Hanger Rod & Shelves, Cinnamon",279.99,359.99,4.0,32.0,80.0,0.22
"iRobot® Roomba® 691 Robot Vacuum – Self Charging, Wi-Fi Connected, Good for Pet Hair",229.99,499.99,4.2,222.0,270.0,0.54
...,...,...,...,...,...,...
"NOMA Indoor Single Tube LED Light Bar with Motion Sensor, 5000 Lumens, 4-ft, Daylight, 50W",82.49,164.99,3.7,9.0,82.5,0.50
"PADERNO Richmond Satin Stainless Steel Flatware Set, Serves 4, 20-pc",49.99,129.99,4.4,39.0,80.0,0.62
OttLite Task Flashlight & Clip Lamp,29.99,39.99,3.7,3.0,10.0,0.25


### Output dataframe

In [63]:
imputed_cantire.to_csv('cantire_df.csv', index=True)

In [91]:
browser.quit()